In [30]:
import Pixie as px
import copy
import Greedy as gr
import Layout

layout = px.read("Instancias/CVS/5-5/data5-5-1.dat")
gr.greedy_solve(layout,10)
print(layout.stacks, layout.steps)

[[2, 22, 14, 20], [21, 19, 11, 9, 8], [7, 24, 3, 17, 6, 5], [15, 18, 16, 12], [25, 4, 13, 10, 23, 1]] 10


In [31]:
def policy(layout, P,):
    perf, moves = perform_bg_move(layout,P)
    if perf:
        #print("perform BG move")
        s_o, s_d = select_bg_move(layout, moves, P)
        layout.dismantling_stack=None
    else: #dismantle stack
        #print("dismantling move")
        s_o = select_dismantle_stack(layout, P)
        s_d = select_destination_stack(layout, s_o, P)
        layout.dismantling_stack = s_o
    return s_o,s_d

In [32]:
def perform_bg_move(layout, P):
    moves = gr.SF_moves(layout)
    
    if len(moves)==0: return False, None
    min_dif, _,_= moves[0]
    min_dif /= layout.max_item
    
    S=len(layout.stacks); H=layout.H
    sorted_stacks = sum(layout.sorted_stack)/S
    av_slots = 0
    av_stacks = 0
    for i in range(len(layout.stacks)):
        if layout.sorted_stack[i]:
            av_slots += H - len(layout.stacks[i])
            if H - len(layout.stacks[i])>0: av_stacks += 1
    av_slots /= H
    av_stacks /= S
    # min stacks que es necesario vaciar para hacer espacio (/S)
    unsorted_items = (layout.total_elements - sum(layout.sorted_elements)) /H
    size_ds=0; rh=0
    if layout.dismantling_stack != None:
        ds = layout.dismantling_stack
        size_ds = len(layout.stacks[ds])/H
        if layout.is_sorted_stack(ds):
            rh = Layout.reachable_height(layout, ds)/H        
        #* características del stack que estaba siendo desmantelado

    #print(P[0] + P[1]*sorted_stacks + P[2]*av_slots + P[3]*av_stacks + P[4]*unsorted_items + P[5]*min_dif)
    return ((P[0] + P[1]*sorted_stacks + P[2]*av_slots + P[3]*av_stacks + P[4]*unsorted_items + P[5]*min_dif + P[6]*size_ds + P[7]*rh) >= 0.5), moves

 


In [33]:
def select_bg_move(layout, moves, P):
    max_value = -np.inf
    best_move = moves[0][1], moves[0][2]
    for m in moves:
        dif = m[0] / layout.max_item
        gv = layout.stacks[m[1]][-1]/layout.max_item
        sizeo = len(layout.stacks[m[1]])/layout.H
        sized = len(layout.stacks[m[2]])/layout.H
        value = P[8]*dif + P[9]*gv + P[10]*sizeo + P[11]*sized
        if value>max_value: 
            max_value=value
            best_move = m[1], m[2]
    return best_move

   
    

In [34]:
def select_dismantle_stack(layout, P):
    H = layout.H
    max_value = -np.inf
    best_stack = None
    for s in range(len(layout.stacks)):
        if len(layout.stacks[s])==0: continue
        ds=0.0
        if(layout.dismantling_stack==s): ds=1.0
        if layout.is_sorted_stack(s):
            size = len(layout.stacks[s])/H
            rh = Layout.reachable_height(layout, s)/H
            prom = np.mean(layout.stacks[s])/layout.max_item
            
            value= P[13] + P[14]*size + P[15]*rh + P[16]*prom + P[17]*ds
        else:
            size = len(layout.stacks[s])/H
            prom = np.mean(layout.stacks[s])/layout.max_item
            # máximo GV no ordenado
            value= P[17] + P[18]*size + P[19]*prom + P[20]*ds
        if value>max_value: 
            max_value=value
            best_stack = s
    return best_stack



In [35]:
def select_destination_stack(layout, s_o, P):
    gv = layout.stacks[s_o][-1]
    H = layout.H
    max_value = -np.inf
    best_stack = None
    for s in range(len(layout.stacks)):
        if len(layout.stacks[s])==H or s==s_o: continue
        if layout.is_sorted_stack(s) and gv <= Layout.gvalue(layout.stacks[s], layout.max_item): #XG
            dif = (Layout.gvalue(layout.stacks[s], layout.max_item) - gv)/layout.max_item
            value = P[21] + P[22]*dif
            
        else: #XB
            dif = (gv - Layout.gvalue(layout.stacks[s], layout.max_item))/layout.max_item
            if(dif < 0): dif = 1 #bad move
            size = len(layout.stacks[s])/H
            if layout.is_sorted_stack(s): sorted=1.0
            else: sorted=0.0

            value= P[23] + P[24]*dif + P[25]*size + P[26]*sorted
        if value>max_value: 
            max_value=value
            best_stack = s
    return best_stack  



In [36]:
import numpy as np

x = np.random.rand(27)
P = x / np.linalg.norm(x)

policy(layout, P,)

(1, 0)

In [37]:
def greedyP(layout, P, max_steps=50, verbose=False):
    if layout.steps>=max_steps: return (max_steps + (layout.total_elements - sum(layout.sorted_elements)))/max_steps
    while layout.unsorted_stacks>0:
        s_o,s_d = policy(layout, P)
        if layout.move(s_o,s_d) == None:
            #print (layout.stacks, s_o, s_d)
            exit()
        if layout.steps>=max_steps: 
            if verbose: print(max_steps,"+",layout.total_elements - sum(layout.sorted_elements))
            return (max_steps + (layout.total_elements - sum(layout.sorted_elements)))/max_steps
    if verbose: print(layout.steps)
    return layout.steps/max_steps

In [38]:
def eval(P):
    ev = 0.0
    for lay in layouts:
        layy=copy.deepcopy(lay)
        ev += greedyP(layy, P)
    return ev/len(layouts)


def evals(PP):
    ret = []
    for P in PP:
        ev = 0.0
        for lay in layouts:
            layy=copy.deepcopy(lay)
            ev += greedyP(layy, P)
        ret.append(ev/len(layouts))
    return np.array(ret)

In [39]:
layouts = []
for i in range(1,21):
    layouts.append(px.read("Instancias/CVS/5-5/data5-5-"+str(i)+".dat"))

#for lay in range(4):
#    gr.greedy_solve(layouts[lay],20)

In [40]:
#lay = copy.deepcopy(layout)
evals([P])

array([1.314])

In [65]:
import pyswarms as ps
dim=27 #size of the vector

# Set-up hyperparameters of PSO
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
PP = np.array(P*100)
PP.shape=(100,27)
print(PP.shape)
#dimensions -> cantidad de dimensiones del vector v
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dim, options=options, init_pos=PP)
optimizer.optimize(evals, iters=100, verbose=True)

2021-07-23 19:26:25,837 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

pyswarms.single.global_best:   0%|                                                                               |0/100

(100, 27)



pyswarms.single.global_best:   0%|                                                               |0/100, best_cost=1.16
pyswarms.single.global_best:   1%|▋                                                              |1/100, best_cost=1.16
pyswarms.single.global_best:   1%|▋                                                              |1/100, best_cost=1.16
pyswarms.single.global_best:   2%|█▎                                                             |2/100, best_cost=1.16
pyswarms.single.global_best:   2%|█▎                                                             |2/100, best_cost=1.16
pyswarms.single.global_best:   3%|█▉                                                             |3/100, best_cost=1.16
pyswarms.single.global_best:   3%|█▉                                                             |3/100, best_cost=1.16
pyswarms.single.global_best:   4%|██▌                                                            |4/100, best_cost=1.16
pyswarms.single.global_best:   4%|██▌  

(1.0554999999999999,
 array([ 0.57844069, -0.32716772,  0.03223434,  0.05842109,  1.71627125,
         0.81982821,  0.47938866,  0.26647778, -0.49194363,  1.51925652,
        -0.3116381 , -0.06084276, -0.37781981,  0.46174141,  0.1242823 ,
        -0.51347006, -0.05770911,  1.1859741 , -1.35191665,  0.22092344,
         0.49474928,  1.34477401, -0.14530816, -0.03349612, -0.49307209,
         0.12095219, -0.07216563]))

In [50]:
layouts = []
for i in range(1,41):
    layouts.append(px.read("Instancias/CVS/5-10/data5-10-"+str(i)+".dat"))

    P=[ 0.51081772, -0.1949631 , -0.72211957,  0.53222943,  1.07463314,
         1.19613466,  0.48256633,  0.53450975,  0.12004768,  1.29994024,
        -0.02021117,  0.06974288, -0.58133095, -0.22831008,  0.43425921,
        -0.04657868,  0.32581445,  1.63571298, -0.93743473,  0.09941223,
         0.28242551,  0.36733418, -0.14236606, -0.23683602,  0.04540262,
         0.37824324,  0.0122877]

for lay in layouts:
    layy=copy.deepcopy(lay)
    greedyP(layy, P, verbose=True)


50 + 8
50 + 8
50 + 11
50 + 7
50 + 12
50 + 10
50 + 10
50 + 6
50 + 2
50 + 8
50 + 6
50 + 7
50 + 11
50 + 9
50 + 7
50 + 12
50 + 11
50 + 5
50 + 6
50 + 9
50 + 7
50 + 11
50 + 2
50 + 7
50 + 3
50 + 10
50 + 7
50 + 14
50 + 8
50 + 5
50 + 5
50 + 7
50 + 10
50 + 9
50 + 8
50 + 7
50 + 9
50 + 8
50 + 8
50 + 6


In [70]:
import numpy as np
from scipy.optimize import minimize

P = np.random.rand(23)

res = minimize(eval, P, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 1.314000
         Iterations: 24
         Function evaluations: 599


In [72]:
res = minimize(eval, P, method='BFGS',
               options={'disp': True})

Optimization terminated successfully.
         Current function value: 1.314000
         Iterations: 0
         Function evaluations: 25
         Gradient evaluations: 1
